# Carga de Datos

## Librerías que se utilizarán

In [1]:
import os
import pandas as pd
import re

### Con el siguiente bloque de código concatemos los path de los archivos csv para después crear un dataframe

In [2]:
#contaminantes_catalogos
path_catalogos = r'/Users/sebastian/PycharmProjects/RegresionLinealSimple/data/contaminantes_catalogos/'
files_catalogos = os.listdir(path_catalogos)
file_list_catalogos = [files_catalogos for files_catalogos in files_catalogos if files_catalogos[:3] == 'cat']
file_list_catalogos

['cat_unidades.csv', 'cat_estacion.csv', 'cat_parametros.csv']

In [3]:
path_csv_cat = []
#path_csv_cat = [os.path.join(path_catalogos, n) for n in file_list_catalogos]
i =0
for file in file_list_catalogos:
    print(file)
    path_csv_cat.append(path_catalogos+file)
    i+=1
df = pd.read_csv(path_csv_cat[0], sep=',',  encoding='latin-1', header=0)
df.head()


cat_unidades.csv
cat_estacion.csv
cat_parametros.csv


,id_unidad,clave_unidad,nombre_unidad
0,1,ppb,partes por billón
1,2,µg/m³,microgramos por metro cúbico
2,3,m/s,metros por segundo
3,4,°,grados
4,5,°C,grados Celsius


In [4]:
df = pd.read_csv(path_csv_cat[1], sep=',',  encoding='latin-1', header=1)
df.head()


,cve_estac,nom_estac,longitud,latitud,alt,obs_estac,id_station
0,ACO,Acolman,-98.912003,19.635501,2198.0,NaN,484150020109
1,AJU,Ajusco,-99.162611,19.154286,2942.0,NaN,484090120400
2,AJM,Ajusco Medio,-99.207744,19.272161,2548.0,NaN,484090120609
3,ARA,Aragón,-99.074549,19.470218,2200.0,Finalizó operación en 2010,484090050301
4,ATI,Atizapan,-99.254133,19.576963,2341.0,NaN,484150130101


In [5]:
df = pd.read_csv(path_csv_cat[2], sep=',',  encoding='latin-1', header=1)
for c in df.columns:
    print( c , df[c].map(type).unique().tolist()  , "\n"*1 )

id_parameter [<class 'int'>] 

cve_param [<class 'str'>] 

nom_param [<class 'str'>] 

unidades_param [<class 'str'>] 



### Limpieza de csvs de carpeta contaminantes_cdmx

In [6]:
#Contaminantes_CDMX
path_cdmx = '/Users/sebastian/PycharmProjects/RegresionLinealSimple/data/contaminantes_CDMX/'
files_cdmx = os.listdir(path_cdmx)
file_list_cdmx = [files_cdmx for files_cdmx in files_cdmx if files_cdmx[:3] == 'con']
file_list_cdmx


['contaminantes_2022.csv',
 'contaminantes_2021.CSV',
 'contaminantes_2020.CSV',
 'contaminantes_2018.CSV',
 'contaminantes_2019.CSV']

In [7]:
path_csv_cdmx = []
i =0
for file in file_list_cdmx:
    print(file)
    path_csv_cdmx.append(path_cdmx+file)
    i+=1

path_csv_cdmx
df = pd.read_csv(path_csv_cdmx[2], sep = ',', encoding='latin-1',  on_bad_lines='skip', header= None)
df.head(15)

contaminantes_2022.csv
contaminantes_2021.CSV
contaminantes_2020.CSV
contaminantes_2018.CSV
contaminantes_2019.CSV


,0
0,Pollutants
1,city: Ciudad de M&eacute;xico
2,cityCode: MEX
3,country: M&eacute;xico
4,mesurementAgency: SIMAT
5,URL: http://www.aire.cdmx.gob.mx
6,timeStamp: 2020/01/01 al 2020/12/31
7,average_interval: 001h
8,version:DT023-IAD-20210803-145717
9,key: M-6L5KR-KG0OT-MBWXE-UXY0F\t


#### Podemos notar que este conjunto de csvs tienen los primeros 10 renglones con información que no sirve, por lo que tenmos que crear nuestro dataframe a partir del renglón 11

In [8]:

#con este for unimos los csv de la carpeta contaminantes_cdmx y se crea el data frame
for i, archivo in enumerate(path_csv_cdmx):
    print(i)
    if i == 0:
        df = pd.read_csv(archivo, sep = ',', encoding='latin-1',  on_bad_lines='skip',skiprows=10,header=0)
    else:
        df = pd.concat([df , pd.read_csv(archivo, sep = ',', encoding='latin-1',  on_bad_lines='skip',skiprows=10,header=0)])


df.info()

0
1
2
3
4


### Para transformar los tipos de datos de  la columna de date, la separamos en dos, fecha y hora

In [74]:
from datetime import datetime
df[['dat_e','time_r']] = df['date'].str.split(' ', n=1,expand=True) #split() separa a partir del caracter que le indiques


In [93]:

#cambio de fehca str a datetime
df.dat_e=pd.to_datetime(df['dat_e'],format="%d/%m/%Y")

'''Para poder convertir las horas a formato datetime tenemos que hacer que los datos que son 24:00 se pasen a 00:00 ya que to_datetime() necesita ese formato'''
for row in df.time_r:
    if row == '24:00':
        df.time_r ='00:00'
    else:continue

df.time_r=pd.to_datetime((df['time_r']))#tranforma las horas de str a datetime
df['time_r'].info()

<class 'pandas.core.series.Series'>
Int64Index: 9931032 entries, 0 to 2266535
Series name: time_r
Non-Null Count    Dtype         
--------------    -----         
9931032 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 216.0 MB
